In [1]:
#Step 1: Install datasets
!pip install datasets==2.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.3 MB/s eta 0:00:00


In [2]:
# Step 2: Load Ms_Marco

from datasets import load_dataset

# dataset = load_dataset("ms_marco", download_mode="force_redownload", split="all")
# dataset = load_dataset("ms_marco", split="all")
dataset = load_dataset("ms_marco", "v1.1",  ignore_verifications=True)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1760: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/v1.1-1729ae2b053254ea/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [3]:
subset = dataset['test']

In [4]:
queries_infos = []
queries = []
corpus = []
for sample in subset:
    query_type = sample['query_type']
    if query_type != 'entity':
        continue
    query_id = sample['query_id']
    query_str = sample['query']
    passages_dict = sample['passages']
    is_selected_lst = passages_dict['is_selected']
    passage_text_lst = passages_dict['passage_text']
    query_info = {
        'query_id': query_id,
        'query': query_str,
        'relevant_docs': []
    }

    current_len_corpus = len(corpus)
    for idx in range(len(is_selected_lst)):
        if is_selected_lst[idx] == 1:
            doc_idx = current_len_corpus + idx
            query_info['relevant_docs'].append(doc_idx)

    if query_info['relevant_docs'] == []:
        continue

    queries.append(query_str)
    queries_infos.append(query_info)
    corpus += passage_text_lst
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')
english_stopwords = stopwords.words('english')
remove_chars = string.punctuation
stemmer = PorterStemmer()

def text_normalize(text):
    text = text.lower()
    for char in remove_chars:
        text = text.replace(char, '')
        text = ' '.join([word for word in nltk.word_tokenize(text) if word not in english_stopwords])
    text = ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(text)])
    return text




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
#build vocabulary:
from nltk import word_tokenize as tokenize

def create_dictionary(corpus):
    dictionary = []
    for doc in corpus:
        normalized_doc = text_normalize(doc)
        tokens = tokenize(normalized_doc)
        for token in tokens:
            if token not in dictionary:
                dictionary.append(token)
    return dictionary



In [8]:
def vectorize(text, dictionary):
    word_count_dict = {word: 0 for word in dictionary}
    tokens = tokenize(text)
    for token in tokens:
        try:
            word_count_dict[token] += 1
        except:
            pass

    vector = list(word_count_dict.values())
    return vector


In [10]:
def create_doc_term_matrix(corpus, dictionary):
    doc_term_matrix = {}
    for idx, doc in enumerate(corpus):
        normalized_doc = text_normalize(doc)
        vector = vectorize(normalized_doc, dictionary)
        doc_term_matrix[(doc, idx)] = vector
    return doc_term_matrix

In [11]:
from scipy import spatial

def similarity(a, b):
    return 1 - spatial.distance.cosine(a, b)


In [12]:
def ranking(query, dictionary, doc_term_matrix):
    normalized_query = text_normalize(query)
    query_vec = vectorize(normalized_query, dictionary)
    scores = []
    for doc_info, doc_vec in doc_term_matrix.items():
        sim = similarity(query_vec, doc_vec)
        scores.append((sim, doc_info))
    scores.sort(reverse=True)
    return scores


In [15]:
dictionary = create_dictionary(corpus)
doc_term_matrix = create_doc_term_matrix(corpus, dictionary)

In [24]:
query_lst = ['What is the official language in Fiji?']
top_k = 10
for query in query_lst:
  print(query)
  scores = ranking(query, dictionary, doc_term_matrix)
  print(scores[:10])
  print(f'Query: {query}')
  print('=== Relevant docs ===')
  for idx in range(top_k):
      doc_score = scores[idx][0]
      doc_content = scores[idx][1][0]
      # print(scores[idx][1][1])

      print(f'Top {idx + 1}; Score: {doc_score:.4f}')
      print(doc_content)
      print('\n')


What is the official language in Fiji?
[(0.6556100681071858, ('The official languages in Fiji are Fijian and English. A dialect of Hindustani is also widely spoken among Indo-Fijians.  _________________________________________   T … he official and everyday language of Fiji is English. Fijian and Fiji-Hindi are second languages in the island nation.', 6388)), (0.6556100681071858, ('The official languages in Fiji are Fijian and English. A dialect of Hindustani is also widely spoken among Indo-Fijians.  _________________________________________   T … he official and everyday language of Fiji is English. Fijian and Fiji-Hindi are second languages in the island nation.', 6386)), (0.6467616667635546, ('The official languages. Fiji’s 1997 Constitution established Fijian as one of the official languages of the country. Fijian is an Austronesian language, a grouping that includes thousands of other languages spanning the globe. The language is of the Malayo-Polynesian family, not too different